# Hyper-Parameter Tuning



There are two important techniques to fine-tune the hyperparameters of the model: Grid Search and Cross Validation.

- Grid Search
  - Define a few parameter values and experiment all these values in modeling. Use `sklearn.model_selection.GridSearchCV` to find the best parameter settings.
- Cross Validation
  - Fine-tune the parameters using cross-validation. Common CV methods include `sklearn.model_selection.StratifiedKFold`, `sklearn_model_selection.ShuffleSplit`, `LeaveOneOut`.

Both mothods can make use of the `pipeline` in sklearn to streamline the processing of training and validation.

## SVM Model

- This example is from the officient `sklearn` documentation
- A classic SVM model with train and test split

In [1]:
## Normal SVM model
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

X, y = datasets.load_iris(return_X_y=True)
X.shape, y.shape

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=0)

X_train.shape, y_train.shape

X_test.shape, y_test.shape

## fit on train sets
clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
## test on test sets
clf.score(X_test, y_test)


0.9666666666666667

## Default K-fold Cross Validation

- We can easily use the `cross_val_score()` to do the K-fold cross validation for a specific training model (without shuffling)

:::{note}
By default, `train_test_split` returns a random split.
:::

In [2]:
## Cross-validation

from sklearn.model_selection import cross_val_score
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(clf, X, y, cv=10, scoring='f1_macro')
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))



Accuracy: 0.97 (+/- 0.09)


## GridSearchCV and Fine-Tuning Hyper-Parameters

- Important steps:
  - Define SVM classifier
  - Define a set of parameter values to experiment
  - Use `GridSearchCV` to find the best parameter settings
  - `GridSearchCV` by default implements cross-validation methods to find the optimal parameter settings.
  - In other words, we can specify our own more sophisticated CV methods in `GridSearchCV`.

In [3]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
iris = datasets.load_iris()
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=5) # 
clf.fit(iris.data, iris.target)


sorted(clf.cv_results_.keys())

#print(clf.cv_results_)

import pandas as pd

cv_results_df=pd.DataFrame(clf.cv_results_)
cv_results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000737,0.000221,0.000469,0.000225,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.000651,0.000035,0.000331,0.000015,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,0.966667,0.966667,0.933333,1.0,0.966667,0.021082,4
2,0.000511,0.000027,0.000276,0.000013,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.000000,0.900000,0.966667,1.0,0.973333,0.038873,3
3,0.000604,0.000047,0.000344,0.000070,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.000000,0.966667,0.966667,1.0,0.980000,0.016330,1


## Cross-validation, Hyper-Parameter Tuning, and Pipeline

- Common cross validation methods:
  - `StratifiedKFold`: Split data into train and validation sets by preserving the percentage of samples of each class
  - `ShuffleSplit`: Split data into train and validation sets by first shuffling the data and then splitting
  - `StratifiedShuffleSplit`: Stratified + Shuffled
  - `LeaveOneOut`: Creating train sets by taking all samples execept one, which is left out for validation
- Important Steps
  - Define all preprocessing methods as Tranasformer
  - Create a pipeline
  - Define a cross-validation method
  - Use `cross_val_score()` to fine the best parameter settings by feeding the pipeline and the CV method

In [4]:
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit

## Create pipeline
clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(C=1))

## Define CV methods
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
shuffsplit = ShuffleSplit(n_splits=5, test_size=0.1, random_state=7)

## CV results
print('K-fold CV:', cross_val_score(clf, X, y, cv=kfold))
print('Shuffle CV:', cross_val_score(clf, X, y, cv=shuffsplit))

K-fold CV: [0.96666667 0.96666667 0.96666667 0.9        1.        ]
Shuffle CV: [0.86666667 1.         1.         1.         0.93333333]


## Deep Learning Example

- This is based on [GridSearchCV with keras](https://www.kaggle.com/shujunge/gridsearchcv-with-keras)

## MNIST Dataset

In [5]:
import numpy as np
import os
from keras.datasets import mnist
from keras.layers import *
from keras.models import *
from time import time
nb_classes = 10
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 784)
X_test = X_test.reshape(X_test.shape[0], 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [6]:
## Convert y labels (integers) into binary class matrix
from keras.utils import np_utils
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)
print(Y_train[:5,])

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## Hyper-Parameter Tuning Using Grid Search

In this example, we aim to fine-tune the following hyper-parameters of the deep neural network:

- `optimizer`
- `kernel_initializer` of the Dense layer

We can fine-tune other common parameters like epochs and batch-sizes.

In [7]:
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    model = Sequential()
    model.add(Dense(512, input_shape=(784,)))
    model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
    model.add(Dropout(0.2))   # Dropout helps protect the model from memorizing or "overfitting" the training data
    model.add(Dense(512, kernel_initializer=init))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, kernel_initializer=init))
    model.add(Activation('softmax')) # This special "softmax" a
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
    return model

In [8]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
start=time()
model = KerasClassifier(build_fn=create_model)
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
# epochs = [50, 100, 150]
# batches = [5, 10, 20]
param_grid = dict(optimizer=optimizers, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, Y_train)
print("total time:",time()-start)


1875/1875 [==============================] - 4s 2ms/step - loss: 0.0370 - accuracy: 0.9340
total time: 163.10951614379883


In [9]:
## The Best Model
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

## Print all experiment results
import pandas as pd
results = pd.DataFrame(grid_result.cv_results_)
results


Best: 0.961483 using {'init': 'glorot_uniform', 'optimizer': 'adam'}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_init,param_optimizer,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,4.962227,0.534804,0.789589,0.021689,glorot_uniform,rmsprop,"{'init': 'glorot_uniform', 'optimizer': 'rmspr...",0.959333,0.965083,0.957750,0.956667,0.963333,0.960433,0.003244,3
1,4.087789,0.024603,0.798292,0.013339,glorot_uniform,adam,"{'init': 'glorot_uniform', 'optimizer': 'adam'}",0.962583,0.960417,0.959250,0.962417,0.962750,0.961483,0.001401,1
2,4.723551,0.063123,0.842619,0.107239,normal,rmsprop,"{'init': 'normal', 'optimizer': 'rmsprop'}",0.959917,0.962083,0.957333,0.956000,0.964333,0.959933,0.003040,4
3,4.174438,0.124374,0.784431,0.015665,normal,adam,"{'init': 'normal', 'optimizer': 'adam'}",0.956833,0.961167,0.961500,0.957333,0.967167,0.960800,0.003712,2
4,4.698246,0.033024,0.798633,0.015098,uniform,rmsprop,"{'init': 'uniform', 'optimizer': 'rmsprop'}",0.961417,0.962083,0.958417,0.947833,0.966250,0.959200,0.006209,5
5,4.086391,0.034679,0.859442,0.142505,uniform,adam,"{'init': 'uniform', 'optimizer': 'adam'}",0.963250,0.951667,0.948333,0.956583,0.954333,0.954833,0.005029,6


## K-fold Validation

- Similar to shuffle split?

In [10]:
np.random.seed(7)
seed=12

In [11]:
def create_model():
    model = Sequential()
    model.add(Dense(512, input_shape=(784,)))
    model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
    model.add(Dropout(0.2))   # Dropout helps protect the model from memorizing or "overfitting" the training data
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Activation('softmax')) # This special "softmax" a
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy']) 
    return model

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
start=time()

model2 = KerasClassifier(build_fn=create_model, epochs=10, batch_size=1500,verbose=1)
results = cross_val_score(model2, X_train, Y_train, cv=10)

print("K-fold Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

print("total time:",time()-start)




Epoch 1/10
36/36 [==============================] - 0s 7ms/step - loss: 0.1047 - accuracy: 0.7911
Epoch 2/10
36/36 [==============================] - 0s 7ms/step - loss: 0.0458 - accuracy: 0.9188
Epoch 3/10
36/36 [==============================] - 0s 7ms/step - loss: 0.0334 - accuracy: 0.9411
Epoch 4/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0255 - accuracy: 0.9559
Epoch 5/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0207 - accuracy: 0.9641
Epoch 6/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0168 - accuracy: 0.9712
Epoch 7/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0141 - accuracy: 0.9755
Epoch 8/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0121 - accuracy: 0.9795
Epoch 9/10
36/36 [==============================] - 0s 5ms/step - loss: 0.0108 - accuracy: 0.9817
Epoch 10/10
4/4 [==============================] - 0s 4ms/step - loss: 0.0121 - accuracy: 0.9782
Epoch 1/10
36/36 [===

## Shuffle Split Cross Validation

- Shuffle but not stratified

In [12]:
## Shuffle Split
start=time()
sf = ShuffleSplit(n_splits=10)
results = cross_val_score(model2, X_train, Y_train, cv=sf)
print("K-fold Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
print("total time:",time()-start)


Epoch 1/10
36/36 [==============================] - 0s 6ms/step - loss: 0.1039 - accuracy: 0.7965
Epoch 2/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0454 - accuracy: 0.9197
Epoch 3/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0335 - accuracy: 0.9410
Epoch 4/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0259 - accuracy: 0.9553
Epoch 5/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0202 - accuracy: 0.9651
Epoch 6/10
36/36 [==============================] - 0s 5ms/step - loss: 0.0169 - accuracy: 0.9713
Epoch 7/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0142 - accuracy: 0.9758
Epoch 8/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0126 - accuracy: 0.9788
Epoch 9/10
36/36 [==============================] - 0s 5ms/step - loss: 0.0098 - accuracy: 0.9839
Epoch 10/10
4/4 [==============================] - 0s 3ms/step - loss: 0.0129 - accuracy: 0.9773
Epoch 1/10
36/36 [===

## Stratified K-Fold Validation


- Stratified but not shuffled
- Stratified cross-validation gives poor results. No idea why? 

In [13]:
print(Y_train[:4,])
print(y_train[:4])
from collections import Counter

Counter(y_train)

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
[5 0 4 1]


Counter({0: 5923,
         1: 6742,
         2: 5958,
         3: 6131,
         4: 5842,
         5: 5421,
         6: 5918,
         7: 6265,
         8: 5851,
         9: 5949})

In [14]:
## Stratified Shuffle
from sklearn.model_selection import StratifiedKFold
start=time()
sk = StratifiedKFold(n_splits=10)
results = cross_val_score(model2, X_train, y_train, cv=sk)
print("Stratified K-fold Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
print("total time:",time()-start)

Epoch 1/10
36/36 [==============================] - 0s 6ms/step - loss: 10.0234 - accuracy: 0.0974
Epoch 2/10
36/36 [==============================] - 0s 5ms/step - loss: 9.8925 - accuracy: 0.1069
Epoch 3/10
36/36 [==============================] - 0s 5ms/step - loss: 9.8916 - accuracy: 0.1004
Epoch 4/10
36/36 [==============================] - 0s 5ms/step - loss: 9.8916 - accuracy: 0.0986
Epoch 5/10
36/36 [==============================] - 0s 6ms/step - loss: 9.8916 - accuracy: 0.1016
Epoch 6/10
36/36 [==============================] - 0s 5ms/step - loss: 9.8916 - accuracy: 0.0997
Epoch 7/10
36/36 [==============================] - 0s 5ms/step - loss: 9.8916 - accuracy: 0.0999
Epoch 8/10
36/36 [==============================] - 0s 5ms/step - loss: 9.8916 - accuracy: 0.1024
Epoch 9/10
36/36 [==============================] - 0s 5ms/step - loss: 9.8916 - accuracy: 0.1017
Epoch 10/10
4/4 [==============================] - 0s 3ms/step - loss: 9.8922 - accuracy: 0.0987
Epoch 1/10
36/36 [==

## Stratified Shuffle Split

- A mix of `StratifiedKFold` and `ShuffleSplit`, which returns stratified randomized folds
- The folds are made by preserving the percentage of samples for each class.

In [15]:
## Stratified Shuffle
from sklearn.model_selection import StratifiedShuffleSplit
start=time()
ss = StratifiedShuffleSplit(n_splits=10)
results = cross_val_score(model2, X_train, Y_train, cv=ss)
print("Stratified Shuffle Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
print("total time:",time()-start)


Epoch 1/10
36/36 [==============================] - 0s 7ms/step - loss: 0.1072 - accuracy: 0.7853
Epoch 2/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0460 - accuracy: 0.9186
Epoch 3/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0341 - accuracy: 0.9406
Epoch 4/10
36/36 [==============================] - 0s 5ms/step - loss: 0.0260 - accuracy: 0.9551
Epoch 5/10
36/36 [==============================] - 0s 5ms/step - loss: 0.0209 - accuracy: 0.9644
Epoch 6/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0170 - accuracy: 0.9708
Epoch 7/10
36/36 [==============================] - 0s 6ms/step - loss: 0.0143 - accuracy: 0.9756
Epoch 8/10
36/36 [==============================] - 0s 5ms/step - loss: 0.0122 - accuracy: 0.9788
Epoch 9/10
36/36 [==============================] - 0s 5ms/step - loss: 0.0101 - accuracy: 0.9828
Epoch 10/10
4/4 [==============================] - 0s 3ms/step - loss: 0.0149 - accuracy: 0.9753
Epoch 1/10
36/36 [===

## GridSearchCV and Cross-Validation

- We perform the `GridSearchCV` to find the optimal optimizers
- We determine the best parameter values based on stratified shuffled 5-fold cross validation

In [16]:
def create_model():
    model = Sequential()
    model.add(Dense(512, input_shape=(784,)))
    model.add(Activation('relu')) # An "activation" is just a non-linear function applied to the output
    model.add(Dropout(0.2))   # Dropout helps protect the model from memorizing or "overfitting" the training data
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10))
    model.add(Activation('softmax')) # This special "softmax" a
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy']) 
    return model
start=time()

## Define cross-validation method
ss = StratifiedShuffleSplit(n_splits=5)

## Define Model
model3 = KerasClassifier(build_fn=create_model)

## Define Hyper-parameter values
optimizers = ['rmsprop', 'adam']
param_grid = dict(optimizer=optimizers, init=init)

## Define GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=ss)
grid_result = grid.fit(X_train, Y_train)
print("total time:",time()-start)


1875/1875 [==============================] - 4s 2ms/step - loss: 0.0384 - accuracy: 0.9313
total time: 203.81905102729797


In [17]:
## The Best Model
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

## Print all experiment results
import pandas as pd
results = pd.DataFrame(grid_result.cv_results_)
results

Best: 0.963000 using {'init': 'uniform', 'optimizer': 'adam'}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_init,param_optimizer,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,6.656565,0.121780,0.524209,0.008416,glorot_uniform,rmsprop,"{'init': 'glorot_uniform', 'optimizer': 'rmspr...",0.960500,0.948500,0.960500,0.962667,0.953333,0.957100,0.005334,6
1,6.022944,0.100696,0.534721,0.013367,glorot_uniform,adam,"{'init': 'glorot_uniform', 'optimizer': 'adam'}",0.960833,0.964833,0.954000,0.957667,0.963333,0.960133,0.003912,4
2,6.614549,0.093743,0.522084,0.007600,normal,rmsprop,"{'init': 'normal', 'optimizer': 'rmsprop'}",0.964667,0.958667,0.964500,0.955667,0.964167,0.961533,0.003693,3
3,5.639868,0.232617,0.496856,0.013943,normal,adam,"{'init': 'normal', 'optimizer': 'adam'}",0.959000,0.958000,0.961333,0.957333,0.963833,0.959900,0.002389,5
4,6.057838,0.092201,0.491186,0.010245,uniform,rmsprop,"{'init': 'uniform', 'optimizer': 'rmsprop'}",0.961833,0.960500,0.959333,0.963833,0.965333,0.962167,0.002178,2
5,5.619616,0.151503,0.484414,0.004364,uniform,adam,"{'init': 'uniform', 'optimizer': 'adam'}",0.961167,0.963500,0.964000,0.960000,0.966333,0.963000,0.002224,1


## References

- [`GridSearchCV` documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)
- [Cross-validation: evaluating estimator performance](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation)
- [Scoring Parameters](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)
- [GridSearchCV with keras](https://www.kaggle.com/shujunge/gridsearchcv-with-keras)

## Requirements

In [18]:
# %run ./get_modules.py
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system/pip names. Unfortunately,
        # there is no systematic way to get pip names from
        # a package's imported name. You'll have to add
        # exceptions to this list manually!
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
        
        
imports = list(set(get_imports()))

# The only way I found to get the version of the root package
# from only the name of the package is to cross-check the names 
# of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

tensorflow==2.3.0
scikit-learn==0.22.2.post1
pandas==1.1.2
numpy==1.18.5
